In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from tsl.data.datamodule import (SpatioTemporalDataModule,
                                 TemporalSplitter)
from tsl.data.preprocessing import StandardScaler
from tsl.metrics.torch import MaskedMAE, MaskedMAPE
from tsl.engines import Predictor
import torch
from src.gnn_dataset import ERA5Dataset
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from src.gat import PFGAT

from src.utils import get_adj_matrix
from pathlib import Path

In [ ]:
adj_mat, target_mapped, fips2idx = get_adj_matrix()
splitter = TemporalSplitter(val_len=0.1, test_len=0.2)
scalers = {'target': StandardScaler(axis=(0, 1))}
data_path = Path("/home/jaydenfassett/powerup") / "data"
# dataset = ERA5Dataset(
#     target_mapped.resample("1h").median(),
#     covariates=None,
#     connectivity=adj_mat,
#     weather_zarr_url="gs://gcp-public-data-arco-era5/co/single-level-reanalysis.zarr-v2",
#     county_shapefile=data_path / "cb_2018_us_county_500k.shp",
#     window=12,
#     horizon=1,
# )
# dm = SpatioTemporalDataModule(
#     dataset=dataset,
#     scalers=scalers,
#     splitter=splitter,
#     batch_size=64,
# )

/home/jaydenfassett/powerup/src/gnn_dataset.py:110: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  self.counties["centroid"] = self.counties.geometry.centroid
/home/jaydenfassett/powerup/src/gnn_dataset.py:112: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  [[lon_to_360(pt.x), pt.y] for pt in self.counties.centroid]
/home/jaydenfassett/powerup/src/gnn_dataset.py:116: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  ds = xr.open_zarr(
/home/jaydenfassett/powerup/src/gnn_dataset.py:116: UserWarning: The specified chunks separate the stored chunks along dimension "t

selecting region
Buiding triangulation
Triangulation done


In [ ]:
# qq = dataset[0]

Interpolating cape
Interpolating d2m
Interpolating hcc
Interpolating istl1
Interpolating istl2
Interpolating istl3
Interpolating istl4
Interpolating lcc
Interpolating mcc
Interpolating msl
Interpolating p79.162
Interpolating p80.162
Interpolating siconc
Interpolating skt
Interpolating sp
Interpolating sst
Interpolating stl1
Interpolating stl2
Interpolating stl3
Interpolating stl4
Interpolating swvl1
Interpolating swvl2
Interpolating swvl3
Interpolating swvl4
Interpolating t2m
Interpolating tcc
Interpolating tciw
Interpolating tclw
Interpolating tcrw
Interpolating tcsw
Interpolating tcw
Interpolating tcwv
Interpolating tsn
Interpolating u10
Interpolating u100
Interpolating v10
Interpolating v100
Interpolating z


/home/jaydenfassett/powerup/src/gnn_dataset.py:186: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cov = torch.tensor(self._interpolate(ds_w))


In [23]:
import pandas as pd
import geopandas as gpd

# 1) Read the raw sources
graph   = pd.read_csv(data_path / "geographic" / "graph.csv", dtype={"src": int, "dest": int})
outages = pd.read_csv(data_path / "eaglei_data" / "eaglei_outages_2022.csv", dtype={"fips_code": int})

# 2) Build the set of all FIPS that appear either as src/dest or in the outages
keep = set(graph["src"]) \
     | set(graph["dest"]) \
     | set(outages["fips_code"])

# 3) Load the full county shapefile and filter to those FIPS
counties = gpd.read_file(data_path / "cb_2018_us_county_500k.shp")
counties["GEOID_int"] = counties["GEOID"].astype(int)
filtered = counties[counties["GEOID_int"].isin(keep)].copy()

# 4) Write out the trimmed shapefile
filtered.to_file("filtered_counties.shp")


In [ ]:
dataset = ERA5Dataset(
    target_mapped.resample("1h").median(),
    covariates=None,
    connectivity=adj_mat,
    fips2idx=fips2idx,
    weather_zarr_url="gs://gcp-public-data-arco-era5/co/single-level-reanalysis.zarr-v2",
    county_shapefile=data_path / "cb_2018_us_county_500k.shp",
    window=12,
    horizon=1,
)

TypeError: cannot directly convert an xarray.Dataset into a numpy array. Instead, create an xarray.DataArray first, either with indexing on the Dataset or by invoking the `to_dataarray()` method.

In [ ]:
dataset

In [9]:
dm.setup()

for q in dm.test_dataloader():
    print(q)
    break

IndexError: Unlabeled multi-dimensional array cannot be used for indexing: time